# opening과 closing 비교

In [ ]:
import cv2
import numpy as np

cap1 = cv2.VideoCapture('videos/video1_opening.mp4')
cap2 = cv2.VideoCapture('videos/video1_closing.mp4')

num = 0
while True:
    ret1, img1 = cap1.read()
    ret2, img2 = cap2.read()

    if num == 200:
        cv2.imshow('opening', img1)
        cv2.waitKey(0)
        
        cv2.imshow('closing', img2)
        cv2.waitKey(0)
        
        cap1.release()
        cap2.release()
        cv2.destroyAllWindows()
        break
    num += 1
    
cap1.release()
cap2.release()
cv2.destroyAllWindows()
    

# 총 frame개수 파악

In [ ]:
cap1 = cv2.VideoCapture('videos/video1.mp4')
cap2 = cv2.VideoCapture('videos/video1_fps20_test.mp4')


print(cap1.get(cv2.CAP_PROP_FRAME_COUNT), cap2.get(cv2.CAP_PROP_FRAME_COUNT) )

# YOLO model apply

In [ ]:
from imageai.Detection import VideoObjectDetection
import os
import time

start = time.time()

execution_path = os.getcwd()

detector = VideoObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "models/yolo.h5"))
detector.loadModel()

video_path = detector.detectObjectsFromVideo(
    input_file_path=os.path.join(execution_path, "videos/video1_opening.mp4"),
    output_file_path=os.path.join(execution_path, "videos/video1_opening_yolo.mp4"),
    frames_per_second=30, log_progress=True)
print(video_path)

print(time.time() - start)

# preprocess와 YOLO model 합치기

In [ ]:
import cv2
import numpy as np
from imageai.Detection import VideoObjectDetection
import os
import time

start = time.time()

cap = cv2.VideoCapture('videos/v1_row.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 30

writer = cv2.VideoWriter('videos/v1.mp4', fourcc, fps, (width, height))
kernel = np.ones((2,2), np.uint8)


while True:
    ret,img = cap.read()

    
    if ret == False:
        break
    
        #img = cv2.resize(img, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_CUBIC)
        #img = cv2.bilateralFilter(img, 9, 30, 30)
        #img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    writer.write(img)
        
        
cap.release()
writer.release()
cv2.destroyAllWindows()

################yolo적용############
execution_path = os.getcwd()

detector = VideoObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "models/yolo.h5"))
detector.loadModel()

video_path = detector.detectObjectsFromVideo(
    input_file_path=os.path.join(execution_path, "videos/v1.mp4"),
    output_file_path=os.path.join(execution_path, "videos/v1_YOLO"),
    frames_per_second=30, log_progress=True)
print(video_path)

print(time.time() - start)

# v1동영상 frame개수 파악

In [ ]:
import cv2
cap1 = cv2.VideoCapture('videos/v2/v2_raw.mp4')
print(cap1.get(cv2.CAP_PROP_FRAME_COUNT) )

# MtCNN으로 얼굴인식후 저장하기

In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from imageai.Detection import VideoObjectDetection
import os
import time
start = time.time()

detector = MTCNN()
cap = cv2.VideoCapture('videos/v1_YOLO(1584).avi')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 30

writer = cv2.VideoWriter('videos/test_yolo_face.mp4', fourcc, fps, (width, height))

while True: 
    ret, frame = cap.read()
    
    if ret == False: 
        break
    
    file = open('videos/face_count.txt', 'a')
    result = detector.detect_faces(frame)
    if result != []:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']
    
            cv2.rectangle(frame,
                          (bounding_box[0], bounding_box[1]),
                          (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                          (0,155,255),
                          2)
    
            cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
        file.write(str(len(result)) + '\n')
    file.close()
    #cv2.imshow('mtcnn', frame)
    #cv2.waitKey(int(1000/fps))
    #writer.write(frame)
        
    #if cv2.waitKey(1) == 13: #13 은 엔터키
     #   break

cap.release()
writer.release()
cv2.destroyAllWindows()

print(time.time() - start)



# yolo와 mtcnn합치기

In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from imageai.Detection import VideoObjectDetection
import os
import time

start = time.time()

cap = cv2.VideoCapture('videos/test.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)*0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)*0.5)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 30

writer = cv2.VideoWriter('videos/test_pre.mp4', fourcc, fps, (width, height))
kernel = np.ones((2,2), np.uint8)


while True:
    ret,img = cap.read()

    
    if ret == False:
        break
    
    img = cv2.resize(img, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_CUBIC)
    img = cv2.bilateralFilter(img, 9, 30, 30)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    writer.write(img)
        
        
cap.release()
writer.release()
cv2.destroyAllWindows()

################yolo적용############
execution_path = os.getcwd()

detector = VideoObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "models/yolo.h5"))
detector.loadModel()

video_path = detector.detectObjectsFromVideo(
    input_file_path=os.path.join(execution_path, "videos/test_pre.mp4"),
    output_file_path=os.path.join(execution_path, "videos/test_YOLO"),
    frames_per_second=30, log_progress=True)
print(video_path)



detector = MTCNN()
cap = cv2.VideoCapture('videos/test_YOLO.avi')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 30

writer = cv2.VideoWriter('videos/test_YOLO_mtcnn.mp4', fourcc, fps, (width, height))

while True: 
        
    ret, frame = cap.read()
    
    if ret == False: 
        break
        
    result = detector.detect_faces(frame)
    
    if result != []:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']
    
            cv2.rectangle(frame,
                          (bounding_box[0], bounding_box[1]),
                          (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                          (0,155,255),
                          2)
    
            cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
      
        writer.write(frame)
        
   
cap.release()
writer.release()
cv2.destroyAllWindows()

print(time.time() - start)
